# Exploratory Analysis: Bookings & Revenue

This notebook demonstrates loading `bookings.xlsx` and `revenue.xlsx`, computing historical analytics, simple Prophet forecasts, and basic recommendations. Change the paths below if your Excel files are located elsewhere.

In [6]:
import sys
from pathlib import Path
# Ensure project root is on sys.path so `import src` works when running this notebook from the notebooks folder
project_root = Path('..').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
from src.analytics import (
    load_data,
    monthly_bookings_and_revenue,
    monthly_revenue_summary,
    utilization_metrics,
    top_tenants,
    top_revenue_products,
    bookings_forecast,
    revenue_forecast,
    recommend_underutilized,
    high_roi_products,
)

base = Path('..').resolve() / 'bookings.xlsx'
revenue = Path('..').resolve() / 'revenue.xlsx'
bookings, revenue = load_data(str(base), str(revenue))
print('Loaded', len(bookings), 'bookings and', len(revenue), 'revenue rows')


c:\Users\dell7\OneDrive\Desktop\Assignment\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


Loaded 676 bookings and 500 revenue rows


In [10]:
# Ensure we are using the latest local code for src.analytics (useful after edits)
import importlib
import src.analytics as _analytics
importlib.reload(_analytics)
# Re-import names into the notebook namespace
from src.analytics import (
    load_data,
    monthly_bookings_and_revenue,
    monthly_revenue_summary,
    utilization_metrics,
    top_tenants,
    top_revenue_products,
    bookings_forecast,
    revenue_forecast,
    recommend_underutilized,
    high_roi_products,
)
print('Reloaded src.analytics; module version reloaded.')


Reloaded src.analytics; module version reloaded.


In [11]:
# Diagnostic: inspect `revenue` DataFrame and run monthly_revenue_summary safely
print('Revenue columns:', list(revenue.columns))
print('\nRevenue sample:')
print(revenue.head().to_string())

# Try calling the summary function and show result or error
try:
    revenue_monthly = monthly_revenue_summary(revenue)
    print('\nmonthly_revenue_summary output sample:')
    print(revenue_monthly.head().to_string())
except Exception as e:
    print('\nmonthly_revenue_summary raised:', type(e).__name__, e)
    # Helpful suggestions
    print('\nSuggestions:')
    print(' - Ensure revenue has a date column (like `date`, `created_at`, or `timestamp`).')
    print(' - Ensure revenue has a price column (like `price`, `amount`, `revenue`, or `calculated_price`).')
    print(' - If column names have leading/trailing spaces, try: revenue.columns = revenue.columns.str.strip()')


Revenue columns: ['charge_type', 'created_at', 'event_at', 'product_type', 'product_name', 'additional_data', 'enterprise_name', 'creator_id', 'user_name', 'price', 'calculated_price', 'calculated_tax', 'surcharge', 'property_name', 'property_timezone', 'refunded', 'status', 'is_deleted', 'product_deleted', 'session_processed', 'result_receipt_number', 'refund_csv_column', 'payment_reference_id']

Revenue sample:
  charge_type                     created_at                       event_at  product_type                                          product_name                                                                                                         additional_data enterprise_name                            creator_id        user_name    price  calculated_price  calculated_tax  surcharge          property_name    property_timezone  refunded    status  is_deleted  product_deleted  session_processed result_receipt_number       refund_csv_column                                     

C:\Users\dell7\OneDrive\Desktop\Assignment\src\analytics.py:86: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = pd.to_datetime(date).dt.to_period('M').dt.to_timestamp()


In [12]:
# Monthly bookings by room/property
bookings_monthly = monthly_bookings_and_revenue(bookings)
bookings_monthly.head()

,month,bookings_count,avg_revenue
0,2022-08-01,3,50.000000
1,2022-09-01,25,192.647059
2,2022-10-01,12,NaN
3,2022-11-01,29,46.428571
4,2022-12-01,18,87.500000


In [13]:
# Monthly revenue summary
revenue_monthly = monthly_revenue_summary(revenue)
revenue_monthly.head()

C:\Users\dell7\OneDrive\Desktop\Assignment\src\analytics.py:86: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = pd.to_datetime(date).dt.to_period('M').dt.to_timestamp()


,month,total_revenue,avg_revenue
0,2022-07-01,110.50,55.250000
1,2022-08-01,10.50,10.500000
2,2022-09-01,221.00,73.666667
3,2022-10-01,32.55,10.850000
4,2022-11-01,92.00,18.400000


In [ ]:
# Seaborn visualizations for monthly summaries
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

# Monthly bookings bar chart
try:
    bm = monthly_bookings_and_revenue(bookings)
    fig, ax = plt.subplots(figsize=(10, 4))
    sns.barplot(data=bm, x='month', y='bookings_count', ax=ax)
    ax.set_title('Monthly Bookings')
    plt.xticks(rotation=45)
    plt.tight_layout()
    display(fig)
except Exception as e:
    print('Could not plot monthly bookings:', e)

# Monthly revenue line chart
try:
    rm = monthly_revenue_summary(revenue)
    fig2, ax2 = plt.subplots(figsize=(10, 4))
    sns.lineplot(data=rm, x='month', y='total_revenue', ax=ax2)
    ax2.set_title('Monthly Revenue')
    plt.xticks(rotation=45)
    plt.tight_layout()
    display(fig2)
except Exception as e:
    print('Could not plot monthly revenue:', e)


In [14]:
# Utilization metrics (9am-6pm)
util = utilization_metrics(bookings, available_start_hour=9, available_end_hour=18)
util.head()

,month,booked_minutes,available_minutes,utilization_pct
0,2022-08-01,90.0,16740,0.537634
1,2022-09-01,840.0,16200,5.185185
2,2022-10-01,240.0,16740,1.433692
3,2022-11-01,180.0,16200,1.111111
4,2022-12-01,60.0,16740,0.358423


In [15]:
# Top tenants and products
print(top_tenants(bookings).head())
print(top_revenue_products(revenue).head())

             enterprise_name  booking_count  total_revenue
17                High Folio            451      32735.220
9               Castle Group             38        661.375
63                spaceandco             21       1062.500
0        08112023_Enterprise             16          0.000
13  Global Occupier Services             14       1849.700
                                          product_name  product_type  \
126                               Waste Wood Recycling  Market place   
62   High Touchpoint Disinfect after regular cleani...  Market place   
2                          Affordable Multi-Faith Room  Meeting room   
105                                        Room No 101   Floor space   
13                                Automation prod test      Balances   

     total_revenue  avg_revenue  count  
126       27300.00  2100.000000     13  
62         9240.00  1026.666667      9  
2          6050.00   504.166667     12  
105        5362.25   109.433673     49  
13      

In [16]:
# Forecast bookings next 3 months
fc_bookings = bookings_forecast(bookings, months=3)
list(fc_bookings.items())[:2]  # show sample keys

16:16:26 - cmdstanpy - INFO - Chain [1] start processing
16:16:36 - cmdstanpy - INFO - Chain [1] done processing
16:16:36 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\dell7\OneDrive\Desktop\Assignment\env\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
c:\Users\dell7\OneDrive\Desktop\Assignment\env\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


[('ds',
  [Timestamp('2022-08-01 00:00:00'),
   Timestamp('2022-09-01 00:00:00'),
   Timestamp('2022-10-01 00:00:00'),
   Timestamp('2022-11-01 00:00:00'),
   Timestamp('2022-12-01 00:00:00'),
   Timestamp('2023-01-01 00:00:00'),
   Timestamp('2023-02-01 00:00:00'),
   Timestamp('2023-03-01 00:00:00'),
   Timestamp('2023-04-01 00:00:00'),
   Timestamp('2023-05-01 00:00:00'),
   Timestamp('2023-06-01 00:00:00'),
   Timestamp('2023-07-01 00:00:00'),
   Timestamp('2023-08-01 00:00:00'),
   Timestamp('2023-09-01 00:00:00'),
   Timestamp('2023-10-01 00:00:00'),
   Timestamp('2023-11-01 00:00:00'),
   Timestamp('2023-12-01 00:00:00'),
   Timestamp('2024-01-01 00:00:00'),
   Timestamp('2024-02-01 00:00:00'),
   Timestamp('2024-03-01 00:00:00'),
   Timestamp('2024-04-01 00:00:00'),
   Timestamp('2024-05-01 00:00:00'),
   Timestamp('2024-06-01 00:00:00'),
   Timestamp('2024-07-01 00:00:00'),
   Timestamp('2024-08-01 00:00:00'),
   Timestamp('2024-09-01 00:00:00'),
   Timestamp('2024-10-01 00:00

In [17]:
# Forecast revenue next 12 months
fc_revenue = revenue_forecast(revenue, months=12)
{ 'ds_sample': fc_revenue['ds'][:5], 'yhat_sample': fc_revenue['yhat'][:5] }

C:\Users\dell7\OneDrive\Desktop\Assignment\src\analytics.py:233: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = pd.to_datetime(date).dt.to_period('M').dt.to_timestamp()
16:16:42 - cmdstanpy - INFO - Chain [1] start processing
16:16:43 - cmdstanpy - INFO - Chain [1] done processing
16:16:43 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\dell7\OneDrive\Desktop\Assignment\env\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
c:\Users\dell7\OneDrive\Desktop\Assignment\env\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


{'ds_sample': [Timestamp('2022-07-01 00:00:00'),
  Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-09-01 00:00:00'),
  Timestamp('2022-10-01 00:00:00'),
  Timestamp('2022-11-01 00:00:00')],
 'yhat_sample': [3155.7051309699646,
  4823.536123209258,
  2570.8898496571264,
  4047.12770527325,
  1965.7119424058287]}

In [18]:
# Recommendations: underutilized rooms and high ROI products
under = recommend_underutilized(util, threshold_pct=30)
roi = high_roi_products(revenue)
under.head(), roi.head()

(        month  booked_minutes  available_minutes  utilization_pct
 14 2023-10-01             0.0              16740         0.000000
 10 2023-06-01             0.0              16200         0.000000
 16 2023-12-01             0.0              16740         0.000000
 4  2022-12-01            60.0              16740         0.358423
 28 2024-12-01            60.0              16740         0.358423,
                                           product_name  product_type  \
 126                               Waste Wood Recycling  Market place   
 81          Marketplace Recurring With Company Invoice  Market place   
 66                                Ility test analytics     Analytics   
 73       Indoor Environment Quality (IEQ) May 30, 2023  Market place   
 62   High Touchpoint Disinfect after regular cleani...  Market place   
 
      total_revenue  avg_rev_per_min  
 126        27300.0      2100.000000  
 81          1500.0      1500.000000  
 66          1200.0      1200.000000  
 